In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import piscola
from astropy.io import fits

In [27]:
hdu = fits.open("../../snpy/snpy/typeIa/Nugent_91bg_SED.fits")
phases = np.arange(-13, 61, 1).astype(int)  # by construction
sed_fluxes = hdu[0].data
header = hdu[0].header
wavelengths = header['CRVAL1'] + (np.arange(header['NAXIS1'], dtype=np.float64) - header['CRPIX1'] + 1) * header['CDELT1']

In [28]:
sed_dict = {'phase':[], 'wave':[], 'flux':[]}
for i, phase in enumerate(phases):
    if phase < -10:
        continue
    fluxes = sed_fluxes[i]
    sed_dict['phase'] =  sed_dict['phase'] + [phase] * len(fluxes)
    sed_dict['wave'] =  sed_dict['wave'] + list(wavelengths)
    sed_dict['flux'] =  sed_dict['flux'] + list(fluxes)
sed_df = pd.DataFrame(sed_dict)
sed_df.astype(float).to_csv('sed_template.dat', sep='\t', index=False, header=False)

In [26]:
with open('README.txt', 'w') as file:
    file.write("1991bg-like SN Ia template taken from SNooPy (Nugent 91bg template).")

In [20]:
interp_dict = {'phase':np.empty(0),
               'wave':np.empty(0),
               'flux':np.empty(0),
              }

waves = sed_df.wave.unique()
phases = sed_df.phase.unique()
interp_phases = np.arange(-13, 60.1, 0.1)

for wave in waves:
    lc_df = sed_df[sed_df.wave==wave]
    lc_phase, lc_flux = lc_df.phase.values, lc_df.flux.values
    interp_flux = np.interp(interp_phases, lc_phase, lc_flux)
    
    interp_wave = np.array([wave]*len(interp_phases))
    interp_dict['phase'] = np.append(interp_dict['phase'], interp_phases)
    interp_dict['wave'] = np.append(interp_dict['wave'], interp_wave)
    interp_dict['flux'] = np.append(interp_dict['flux'], interp_flux)
    
interp_df = pd.DataFrame(interp_dict)
interp_df.sort_values(['phase', 'wave'], inplace=True)

# apply offset to phase to have epochs with respect to B-band peak
Bessell_B = piscola.filters_class.SingleFilter('Bessell_B', 'VEGA')

fluxes_B = []
for phase in interp_df.phase.unique():
    phase_df = interp_df[interp_df.phase==phase]
    flux = Bessell_B.integrate_filter(phase_df.wave.values, phase_df.flux.values)
    fluxes_B.append(flux)
fluxes_B = np.array(fluxes_B)

peak_id = np.argmax(fluxes_B)
peak_offset = interp_df.phase.unique()[peak_id]

interp_df.phase = interp_df.phase.values - peak_offset

In [22]:
peak_offset

-4.618527782440651e-14